# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [101]:
# Dependencies and Setup
# !pip install cartopy geoviews pyproj
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import json

# Import API key
from api_keys import geoapify_key

In [102]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,15.14,85,100,15.40,SH,1693047401
1,1,thompson,55.7435,-97.8558,7.09,93,0,1.54,CA,1693047226
2,2,adamstown,-25.0660,-130.1015,18.69,64,82,6.82,PN,1693047167
3,3,bredasdorp,-34.5322,20.0403,13.48,49,66,6.63,ZA,1693047414
4,4,albany,42.6001,-73.9662,19.77,94,49,2.07,US,1693047211


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [103]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points("Lng","Lat",geo = True,tiles = "OSM",size = "Humidity",scale =0.8,color = "City")

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [105]:
# Narrow down cities that fit criteria and drop any results with null values
max_Temp=25
min_temp=15
Countries = ["US", "AU", "CA","Fr", "ES","RU","IR"]
filtered_cities=city_data_df[(city_data_df['Max Temp'] <= max_Temp) &(city_data_df['Max Temp'] >= min_temp)&(city_data_df['Country'].isin(Countries))]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Display sample data
filtered_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,albany,42.6001,-73.9662,19.77,94,49,2.07,US,1693047211
10,10,bethel,41.3712,-73.4140,22.20,89,100,0.00,US,1693047417
17,17,santa marta de ortigueira,43.6833,-7.8500,18.47,82,65,1.79,ES,1693047870
24,24,houlton,46.1262,-67.8403,17.23,95,100,0.00,US,1693047873
26,26,udachny,66.4167,112.4000,24.14,33,6,4.60,RU,1693047508
...,...,...,...,...,...,...,...,...,...,...
558,558,homer,59.6425,-151.5483,15.94,82,100,3.60,US,1693048068
576,576,newcastle,-32.9272,151.7765,16.51,83,18,2.06,AU,1693048064
578,578,terrace,54.5163,-128.6035,15.05,80,0,0.37,CA,1693048074
579,579,kargasok,59.0556,80.8572,16.47,60,46,3.58,RU,1693047487


### Step 3: Create a new DataFrame called `hotel_df`.

In [106]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns_to_copy = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df = filtered_cities[columns_to_copy].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
4,albany,US,42.6001,-73.9662,94,
10,bethel,US,41.3712,-73.4140,89,
17,santa marta de ortigueira,ES,43.6833,-7.8500,82,
24,houlton,US,46.1262,-67.8403,95,
26,udachny,RU,66.4167,112.4000,33,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [107]:
# Set parameters to search for a hotel
limit = 1
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude= row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params = {
    "categories": ["accommodation.hotel"],
    "limit":limit,    
    "apiKey":geoapify_key
    }
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty/# Convert the API response to JSON format
    response = requests.get(base_url, params=params).json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
santa marta de ortigueira - nearest hotel: Hotel la Perla
houlton - nearest hotel: Shiretown Inn & Suites
udachny - nearest hotel: Вилюй
camden - nearest hotel: Wingate by Wyndham
cannonvale - nearest hotel: Whitsundays Rainforest Retreat
yellowknife - nearest hotel: No hotel found
racine - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
hawaiian paradise park - nearest hotel: No hotel found
yeppoon - nearest hotel: Blossom's on Seaspray
yuzhno-kurilsk - nearest hotel: Dom Druzhby
jamestown - nearest hotel: DoubleTree Jamestown
fisterra - nearest hotel: Hotel Temático do Banco Azul
laguna - nearest hotel: Holiday Inn Express & Suites
palmer - nearest hotel: No hotel found
portland - nearest hotel: the Hoxton
slavgorod - nearest hotel: Теремок
westport - nearest hotel: Norwalk Inn and Conference Center
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central

,City,Country,Lat,Lng,Humidity,Hotel Name
4,albany,US,42.6001,-73.9662,94,No hotel found
10,bethel,US,41.3712,-73.4140,89,Hampton Inn Danbury
17,santa marta de ortigueira,ES,43.6833,-7.8500,82,Hotel la Perla
24,houlton,US,46.1262,-67.8403,95,Shiretown Inn & Suites
26,udachny,RU,66.4167,112.4000,33,Вилюй
...,...,...,...,...,...,...
558,homer,US,59.6425,-151.5483,82,Driftwood Inn & Homer Seaside Lodges
576,newcastle,AU,-32.9272,151.7765,83,Rydges Newcastle
578,terrace,CA,54.5163,-128.6035,80,Sandman Hotel Terrace
579,kargasok,RU,59.0556,80.8572,60,Титан


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [108]:
%%capture --no-display

# Configure the map plot
map_plot_2 =  hotel_df.hvplot.points("Lng","Lat",geo = True,tiles = "OSM",size = "Humidity",scale =0.8,color = "Country",hover_cols=["Hotel Name", "Country"],tools=["hover"])

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,Hotel Name)